In [12]:
from tqdm.notebook import tqdm
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import Button, Layout
from dotenv import load_dotenv
from datetime import datetime
from pathlib import Path
from time import sleep
import pandas as pd
import os

In [13]:
# alpaca_api = 0

# Alpaca connection setup 

def app_init():
    global alpaca_api
    load_dotenv()

    sp500_tickers_df = pd.read_csv(Path("dataset/symbols.csv"), header=None)
    #display(sp500_tickers_df.head(3))
    #display(sp500_tickers_df.tail(3))
    sp500_tickers = sp500_tickers_df.loc[:,0]

    # Setting the variables for the Alpaca API and secret keys
    alpaca_api_key = os.getenv('ALPACA_API_KEY')
    alpaca_api_secret_key = os.getenv('ALPACA_SECRET_KEY')

    #display(type(alpaca_api_key))
    #display(type(alpaca_api_secret_key))

    # Create the Alpaca tradeapi.REST object
    alpaca_api = tradeapi.REST(alpaca_api_key, alpaca_api_secret_key, api_version = "v2")

    return(sp500_tickers, alpaca_api)


In [14]:
# If date range is dropdown-selectable, then will use given end date,
# calculate back three years to get start_date
# instead of relying on values from app_init()

def get_ticker_prices(tickers, num_days=1000):
    """
    Use Alpacas to get ticker price data.

    Parameters:
        tickers: list of stock tickers
        days: days of historical data to get (limit 1000)

    Return value:  DataFrame of ticker price data
    """ 
    
    # Set start and end datetimes between now and num_days ago

    timeframe="1D"
    today=pd.Timestamp.now()
    start_date = pd.Timestamp(today - pd.Timedelta(days=num_days), tz="America/New_York").isoformat()
    end_date = pd.Timestamp(today).isoformat()
    # Set number of rows to three years of days
    limit_rows = 1000

    # Get 3 years’ worth of historical price data for Microsoft and Coca-Cola
    prices_df = alpaca_api.get_barset(
        tickers,
        timeframe,
        start=start_date,
        end=end_date,
        limit=limit_rows
    ).df

    return(prices_df)
   

In [15]:
# Function loading historical data for analysis

def load_data():
    #Loading 3 year data from Alpaca
    
    global start_date
    # getting prices for selected stocks 
    chosen_tickers_df=get_ticker_prices(screened_ticker_list, 1095)

    # Slicing the dataframe to chose only closing prices
    closing_prices_df=chosen_tickers_df.loc[:, (slice(None), 'close')]
    closing_prices_df.columns = closing_prices_df.columns.droplevel(-1)

    #droping time component of the date index ans NAs
    closing_prices_df.index=pd.to_datetime(closing_prices_df.index).strftime("%Y-%m-%d")
    closing_prices_df.dropna()
    return (closing_prices_df)
 

In [16]:
# https://stackoverflow.com/questions/48775841/pandas-ema-not-matching-the-stocks-ema
# ta.ema shown at the end of that page.


def add_EMA(price, day):
    return price.ewm(span=day).mean()

# function to check for EMA crossing
def check_EMA_crossing(df):
    # condition 1: EMA18 is higher than EMA50 at the last trading day
    cond_1 = df.iloc[-1]['EMA18'] > df.iloc[-1]['EMA50']
    # condition 2: EMA18 is lower than EMA50 the previous day
    cond_2 = df.iloc[-2]['EMA18'] < df.iloc[-2]['EMA50']
    # condition 3: to filter out stocks with less than 50 candles
    cond_3 = len(df.index) > 50 
    # will return True if all 3 conditions are met
    return (cond_1 and cond_2 and cond_3)

In [17]:
# If date range is dropdown-selectable, then need to pass start and end dates
# to get_ticker_prices, instead of relying on values from app_init()

def screen_tickers(ticker_list):
    """
    Screen S&P tickers by Exponential Moving Average
    
    Parameters: ticker_list 
    
    Returns: the screened list of tickers
    """
    
    # Preliminary ticker list after screening based on EMA
    # Step 1: get all sp500 prices
    
    sp500_df = get_ticker_prices(ticker_list[0:1], 252)
    done = False
    ticker_count = len(ticker_list) -1
    start=1
    end=101
    # each dataframe has 100 tickers, concat them
    printmd("## Getting Alpacas ticker data:")

    while not done:
        partial_df = get_ticker_prices(ticker_list[start:end], 252)
        sp500_df = pd.concat([sp500_df, partial_df], axis=1)
        start +=100
        end += 100
        if end >= 602:
            done = True

    #display(sp500_df)
    
    screened_list = [] 
    stock_list = ticker_list

    printmd(f"## Screening {ticker_count} S&P 500 tickers:")
    for each_stock in tqdm(stock_list, leave=False):
        ##
        ticker_df = sp500_df[each_stock]
        close_df = pd.DataFrame(columns = ['close'])
        close_df['close'] = ticker_df['close']
        ##
        
        # Step 2: add technical indicators (in this case EMA)
        close_df['EMA18']=add_EMA(close_df['close'],18)
        close_df['EMA50']=add_EMA(close_df['close'],50)
        close_df['EMA100']=add_EMA(close_df['close'],100)
        # if all 3 conditions are met, add stock into screened list
        if check_EMA_crossing(close_df):
            screened_list.append(each_stock)
        #sleep(0.2)

    return (screened_list)


In [18]:
def risk_return_analysis():
    # Loading data
    closing_prices_df=load_data()
    # display(closing_prices_df.head())
    trading_year=252
    
    # Normalizing prices and Plotting the normalized closing prices
    for i in closing_prices_df.columns:
        closing_prices_df.loc[:,i]=closing_prices_df.loc[:,i]/closing_prices_df[i].iloc[0]
    # print("Price chart comparison for chosen tickers")
    fig, axes = plt.subplots(2,2)
#    closing_prices_df.plot(title="Normalized price plot",figsize=(10,5),ax=axes[1,0])

    # Calculating daily returns
    daily_returns_df=closing_prices_df.pct_change().dropna()
    # display(daily_returns_df.head())
    # Visualizing Daily Returns in line and box subplots

    daily_returns_df.plot(kind="line",title="Daily Returns forselected stocks in 3 years",xlabel="Date",ylabel="Daily returns",figsize=(20,10),ax=axes[0,0])
    daily_returns_df.plot(kind="box",title="Daily return distribution",figsize=(20,10),ax=axes[0,1])

    # Calculate and plot the cumulative returns
    # Reviewing the last 5 rows of the cumulative returns DataFrame

    cumulative_returns=(1+ daily_returns_df).cumprod()
    cumulative_returns.tail()
    cumulative_returns.plot(title="Cumulative returns for selected stocks in 3 years",ax=axes[1,0])

    std_dev=closing_prices_df.std()
    #print(f"Calculated standard deviation is \n{std_dev}")

    # Calculating and sort the annualized standard deviation for 252 trading days 
    # Reviewing the annual standard deviations smallest to largest
    
    annualized_std_dev=daily_returns_df.std()* trading_year **1/2
    annualized_std_dev=annualized_std_dev.sort_values()
    #print(f"Calculated annualized standard deviation is \n{annualized_std_dev}")

    # Output to the user as less volatile stocks

    min_3_annualized_st_dev=annualized_std_dev.iloc[0:3]
    #print(f" The 3 less volatile stocks based on annualized standard deviation \n{min_3_annualized_st_dev}")
    #print("The 3 less volatile stocks based on annualized standard deviation:")
    least_volatile_df = pd.DataFrame(min_3_annualized_st_dev)
    #df = least_volatile_df.rename(columns={'0':'st_dev'})
    least_volatile_df.columns = [''] * len(least_volatile_df.columns)
    #display(least_volatile_df)
    
    #Calculating annual average return
    annual_avg_return=daily_returns_df.mean() * trading_year

    # Shows 3 best performing stocks in terms of risk/returns

    # Calculate the annualized Sharpe Ratios for each
    # Review the Sharpe ratios sorted lowest to highest
    sharpe_ratio=annual_avg_return/annualized_std_dev
    sharpe_ratio=sharpe_ratio.sort_values(ascending=False)
    sharpe_ratio.plot.bar(title="Sharpe ratio barchart for selected stocks",ax=axes[1,1])
    best3_returns_per_risk=sharpe_ratio.iloc[0:3]
    #print(f" The best performing 3 stocks based on sharpe ratio \n{best3_returns_per_risk}")
    #print("The best performing 3 stocks based on sharpe ratio")
    best_risk_df = pd.DataFrame(best3_returns_per_risk)
    best_risk_df.columns = [''] * len(best_risk_df.columns)
    #display(best_risk_df)
    
    #Creating an optimal list of tickers
    optimal_tickers = []
    optimal_tickers.append(best3_returns_per_risk.index[0])
    optimal_tickers.append(best3_returns_per_risk.index[1])
    optimal_tickers.append(best3_returns_per_risk.index[2])


    #display(test_df)
    plt.show()
    printmd("---")
    printmd("### The 3 less volatile stocks based on annualized standard deviation:")
    display(least_volatile_df)
    printmd("### The best performing 3 stocks based on sharpe ratio:")
    display(best_risk_df)

    return(optimal_tickers)




In [19]:
# Eventually these belong in the app_init() function

# Setting up weight dictionaries 
MC_median_df_dict = {}
MC_sim_df_dict = {}


# weights_dict = {
#     0:[.33,.33,.33],
#     1:[.10,.30,.60],
#     2:[.60,.30,.10],
#     3:[.20,.40,.40],
#     4:[.40,.40,.20],
#     5:[.20,.60,.20],  
#     6:[.20,.50,.30],
#     7:[.30,.20,.50]
# }
weights_dict3 = {0:[.33,.33,.33],1:[.10,.30,.60],2:[.60,.30,.10],3:[.20,.40,.40],4:[.40,.40,.20],5:[.20,.60,.20],6:[.20,.50,.30],7:[.30,.20,.50]}
weights_dict2 = {0:[.50,.50],1:[.40,.60],2:[.60,.40],3:[.30,.70],4:[.70,.30],5:[.20,.80],6:[.80,.20]}
weights_dict1 = {0:[1.0]}
weights_list = [weights_dict1, weights_dict2, weights_dict3]

In [20]:
# Function generating MC simulations and finding median
def run_MC(tickers_df, sim_duration, sim_count, weights_dict):
    global MC_median_df_dict
    
    for i in range(len(weights_dict)):
        MC_sim_config = MCSimulation(
            portfolio_data = tickers_df,
            weights = weights_dict[i],
            num_simulation = sim_count,
            num_trading_days = sim_duration
        )

        MC_sim = MC_sim_config.calc_cumulative_return()
        MC_sim_median = MC_sim.median(axis=1)
        MC_median_df_dict[i] = MC_sim_median
        
    return(MC_sim_config)


In [21]:
# The start of the application

def main():
    global weights_dict
    global screened_ticker_list
    
    
    plot_output = widgets.Output()
    
    # Get all sp500 tickers
    ticker_list, alpaca_api = app_init()

    # Run screening to get preferred short list
    screened_ticker_list = screen_tickers(ticker_list)
    
    # For Demo, if subset of stocks is less than 3, the application might have to use sample data 
    if len(screened_ticker_list) < 3:
        printmd("### &emsp; Due to market conditions and screening method, using demo list of tickers. Please see README")
        screened_ticker_list = ['AEE', 'DISCK', 'NEM', 'NLSN', 'WYNN']

    # risk_return_analysis uses screened_ticker_list globally
    screened_ticker_list = risk_return_analysis()
    
    if len(screened_ticker_list) < 3:
        weights_dict = weights_list[len(screened_ticker_list) -1]
    else:
        weights_dict = weights_dict3

    #display(print("weights_dict: ", weights_dict))

    #display(screened_ticker_list)
    # Analyze to reduce short list to optimal list
    # optimal_ticker_list = analyze_tickers(screened_ticker_list)

    # until analyze_tickers() exists, take just first three of list
    #test_list = screened_ticker_list[:3]
    #screened_df = get_ticker_prices(optimal_ticker_list, 252)
    optimal_df = get_ticker_prices(screened_ticker_list, 1000)
    
    printmd("---")
    printmd(f"## Running {len(weights_dict)} Monte Carlo simulations, varying weights for portfolio {screened_ticker_list}:")
    MC_sim_class = run_MC(optimal_df, MC_sim_duration, MC_sim_count, weights_dict)

    fig, axes = plt.subplots(1,3, figsize=(25,10))
    with plot_output:
        for key in MC_median_df_dict:
            MC_median_df_dict[key].plot(ax=axes[0], legend=True, label=str(weights_dict[key]), title="Median cumulative returns by security weights")
        
        
        max_value_column=pd.DataFrame(MC_median_df_dict).iloc[-1].idxmax(axis=1)
        label_list = []
        for i in range(len(screened_ticker_list)):
            #print(screened_ticker_list[i] + " " + str(int(weights_dict3[max_value_column][i]  * 100)) + "%")
            label_list.append(screened_ticker_list[i] + " " + str(int(weights_dict3[max_value_column][i]  * 100)) + "%")

        #print(label_list)
        ax = pd.DataFrame(weights_dict3[max_value_column]).plot.pie(ax=axes[1],y=0,title='Best weights for the curent selection will be', labels=label_list)
        ax.legend(loc='upper left')
        
        MC_sim_dist_plot = MC_sim_class.plot_distribution()
        
        plt.show()

    display(plot_output)    


In [22]:
from IPython.display import Markdown, display
from IPython.display import clear_output

# Output

MC_sim_duration = 252
MC_sim_count = 200


def printmd(string):
    display(Markdown(string))
    
def menu_sim_duration_eventhandler(change):
    set_sim_duration(change.new)
    
def menu_sim_count_eventhandler(change):
    set_sim_count(change.new)

output = widgets.Output()
plot_output = widgets.Output()
output.clear_output()

# convert sim_time to days
menu_sim_duration = widgets.Dropdown(options =  ['1 year', '2 years', '3 years'])

# number of simulations to run
menu_sim_count = widgets.Dropdown(options = ['200', '300', '400', '500'])
#bounded_num = widgets.BoundedFloatText(min=0, max=100000, value=5, step=1)

btn = widgets.Button(description='Run S&P 500 Portfolio Advisor', layout=Layout(width='auto'))
btn.style.button_color = 'lightgreen'

menu_sim_duration.observe(menu_sim_duration_eventhandler, names='value')
menu_sim_count.observe(menu_sim_count_eventhandler, names='value')

@output.capture()
def btn_eventhandler(obj):
    clear_output()
    #print('Hello from the {} button!'.format(obj.description))
    main()

btn.on_click(btn_eventhandler)

@output.capture()
def set_sim_duration(value):
    global MC_sim_duration
    if value == '1 years':
        MC_sim_duration = 252
    elif value == '2 years':
        MC_sim_duration = 504
    elif value == '3 years':
        MC_sim_duration = 756

@output.capture()
def set_sim_count(value):
    global MC_sim_count
    MC_sim_count=int(value)

printmd("---")
printmd("# <font color='blue'>Portfolio Advisor:</font>")
printmd("## &emsp; Select a portfolio by screening S&P 500 by EWMA, filtering with TA")
printmd("## &emsp; &emsp; &emsp; &emsp; and running Monte Carlo simulations to security weights.")
printmd("---")
printmd("## Choose duration for Monte Carlo simulations:")
display(menu_sim_duration)
printmd("## Choose interations for Monte Carlo simulations:")

display(menu_sim_count)
display(btn)

display(output)
display(plot_output)

---

# <font color='blue'>Portfolio Advisor:</font>

## &emsp; Select a portfolio by screening S&P 500 by EWMA, filtering with TA

## &emsp; &emsp; &emsp; &emsp; and running Monte Carlo simulations to security weights.

---

...

## Choose interations for Monte Carlo simulations:

Dropdown(options=('200', '300', '400', '500'), value='200')

Button(description='Run S&P 500 Portfolio Advisor', layout=Layout(width='auto'), style=ButtonStyle(button_colo…

Output()

Output()